In [74]:
#dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

In [75]:
#connect to database for actual queries using SQLalchemy
#dependency
from sqlalchemy import create_engine

#set up post gress string
database_path = "healthcare-fraud-db.cakair5uyevn.us-east-1.rds.amazonaws.com"
port = '5432'
user = 'root'
password = 'rootroot'
database_name = 'healthcare-fraud-db'

#postgres string
pgString = ('postgresql://{user}:{password}@{ipaddress}:{port}/{dbname}'.\
           format(user = user,\
                 password = password,\
                 ipaddress = database_path,\
                 port = port,\
                 dbname = database_name))

#create engine for connection
engine = create_engine(pgString)

In [76]:
#lets make sure this worked
data =  engine.execute("SELECT * FROM final_table")
#convert data into a data frame to be able to call and do machine learning
df = pd.read_sql("SELECT * FROM final_table", engine)
df.head()

,npi,is_fraud,total_claims,total_drug_cost,provider_last_name,provider_first_name,provider_city,provider_state,total_payment
0,1730115106,0,61,1629.38,GAJJAR,AAKASH,GALVESTON,TX,101.34
1,1508975772,0,655,67327.96,ABBAS,AAMER,EL PASO,TX,196.21
2,1134439581,0,29888,1902530.64,CLARK,AARON,STATESBORO,GA,1028.72
3,1831310770,0,9968,544819.16,HANSHAW,AARON,DAYTON,OH,65.96
4,1306072038,0,67,298.71,INSEL,AARON,FARMINGTON,CT,2280.76


In [ ]:
#write data frame to csv for easy access to make visuals
df.to_csv("Resources/finalTable.csv")

In [130]:
#dependencies for logistic regression (split, train, )
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#set X and y
X = df.iloc[:,0]
#change from series to array
X = X.to_numpy()
#reshape to fit regression model without changing data
X = X.reshape(-1,1)
y = df.iloc[:,1]
y.nunique()

#Step 1: split to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=27)


In [131]:
#create logistic regression model
classifier = LogisticRegression()
classifier
#extra models
# SVC_model = svm.SVC()
KNN_Model = KNeighborsClassifier(n_neighbors = 5)

In [132]:
#Step 2: fit/train using training data
classifier.fit(X_train, y_train)
# print(type(X_train))
KNN_Model.fit(X_train, y_train)

C:\Users\tiania\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [133]:
#Validate data using test data
print(f"Training Data Score: {classifier.score(X_train, y_train)} KNN Train: {KNN_Model.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)} KNN Test: {KNN_Model.score(X_test, y_test)}")

Training Data Score: 0.9860431128973328 KNN Train: 0.9860431128973328
Testing Data Score: 0.9838049778383907 KNN Test: 0.9838049778383907


In [134]:
#Step 3: test
#make predictions
predictions = classifier.predict(X_test)
KNN_prediction = KNN_Model.predict(X_test)
print(f"First 10 Predictions:   {predictions[:20]}")
print(f"First 10 Actual labels: {y_test[:20].tolist()}")

First 10 Predictions:   [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
First 10 Actual labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [137]:
#show predictions
predictionDF = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
KNNpredictionDF = pd.DataFrame({"Prediction": KNN_prediction, "Actual": y_test}).reset_index(drop=True)
predictionDF.nunique()

Prediction    1
Actual        2
dtype: int64